In [6]:
# file related
import os
from os.path import join
import datetime

# machine learning
from keras.models import Sequential
from keras.layers import Dense, Input
import tensorflow as tf
from sklearn.model_selection import train_test_split

# utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from IPython.display import Audio as play_audio
import scipy as sp
import taunet_utils
import json

# RTNeural special import
import sys
sys.path.append("../RTNeural/python/")
from model_utils import save_model

print(f"TF version {tf.__version__}")

TF version 2.12.0


In [7]:
dataset = "AITD_Dataset_Luka_1"

with open(join("dataset", "saved", dataset + "in.json"), "r") as json_file:
    input_data = json.load(json_file)
with open(join("dataset", "saved", dataset + "out.json"), "r") as json_file:
    output_data = json.load(json_file)

for d in input_data:
    print(f"{d}: {input_data[d]} -> {output_data[d]}")

1 4397-spicy-drums: [0.7748, 0.0379, 0.1278, 1.0, 0.0989, 0.5959, 0.1791, 0.0128, 0.4533, 0.5585, 0.9841, 0.3708, 0] -> [0.7964052, 0.7699047666666667]
1 h_shift_48000_-1: [0.7769, 0.0992, 0.1997, 0.4652, 0.1986, 0.5277, 0.1679, 0.0353, 0.6249, 0.5196, 0.9841, 0.3708, 0] -> [0.7964052, 0.7699047666666667]
1 ch_shift_48000_1: [0.7748, 0.1027, 0.2619, 0.5236, 0.3526, 0.5535, 0.2235, 0.0663, 0.7289, 0.2571, 0.9841, 0.3708, 0] -> [0.7964052, 0.7699047666666667]
1 und_mix_0_9_None: [0.7748, 0.036, 0.2075, 0.5783, 0.1058, 0.6381, 0.1811, 0.0738, 0.4201, 0.6094, 0.9841, 0.3708, 0] -> [0.7964052, 0.7699047666666667]
1 s_time_fade_1_in: [0.7748, 0.0396, 0.1233, 1.0, 0.1052, 0.632, 0.1825, 0.0126, 0.4627, 0.0, 0.9841, 0.3708, 0] -> [0.7964052, 0.7699047666666667]
1 _time_fade_1_out: [0.7748, 0.0392, 0.2652, 0.5251, 0.1112, 0.6503, 0.1966, 0.0734, 0.5223, 0.4176, 0.9841, 0.3708, 0] -> [0.7964052, 0.7699047666666667]
1 time_offset_-0_5: [0.7748, 0.0396, 0.1241, 1.0, 0.1028, 0.642, 0.2214, 0.0131, 

In [8]:
def create_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Dense(32, activation='relu'))
    # model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    # model.add(layers.LSTM(64, return_sequences=True))   # short term memory, useful if input data is related accross vectors
    model.add(Dense(2, kernel_regularizer=tf.keras.regularizers.l2(0.001))) # no activation (linear): continuous mapping of outputs (this is not a classification task!)
    return model

input_shape = (13,)
model = create_model(input_shape)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 32)                448       
                                                                 
 dense_7 (Dense)             (None, 32)                1056      
                                                                 
 dense_8 (Dense)             (None, 16)                528       
                                                                 
 dense_9 (Dense)             (None, 8)                 136       
                                                                 
 dense_10 (Dense)            (None, 4)                 36        
                                                                 
 dense_11 (Dense)            (None, 2)                 10        
                                                                 
Total params: 2,214
Trainable params: 2,214
Non-traina

In [9]:
combined_data = [(input_data[key], output_data[key]) for key in input_data.keys()]

train_data, test_data = train_test_split(combined_data, test_size=0.05, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/" + now
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

class LogRMSECallback(tf.keras.callbacks.Callback):
    def __init__(self, log_dir):
        super(LogRMSECallback, self).__init__()
        self.log_dir = log_dir

    def on_epoch_end(self, epoch, logs=None):
        mse = logs.get('val_loss')
        rmse = np.sqrt(mse)
        with tf.summary.create_file_writer(self.log_dir).as_default():
            tf.summary.scalar('val_rmse', rmse, step=epoch)
        with open(join(self.log_dir, "..", "stats.md"), 'w') as f:
            s = f"# Taunet {now} ({dataset}) stats\n"
            s += f"- MSE: {mse}\n"
            s += f"- RMSE: {rmse}\n"
            f.write(s)

%load_ext tensorboard
%tensorboard --logdir ./logs/ --port 6006

history = model.fit(
    x=np.array([item[0] for item in train_data]),
    y=np.array([item[1] for item in train_data]),
    validation_data=(
        np.array([item[0] for item in val_data]),
        np.array([item[1] for item in val_data])
    ),
    epochs=80,
    # callbacks=[tensorboard_callback, early_stopping_callback],
    callbacks=[tensorboard_callback, early_stopping_callback, LogRMSECallback(join(log_dir, "rmse"))],
    # batch_size=16
)

Reusing TensorBoard on port 6006 (pid 11444), started 7 days, 23:19:43 ago. (Use '!kill 11444' to kill it.)

Epoch 1/80
20/20 [==============================] - 2s 27ms/step - loss: 0.3742 - accuracy: 0.5217 - val_loss: 0.2780 - val_accuracy: 0.5577
Epoch 2/80
20/20 [==============================] - 0s 12ms/step - loss: 0.2564 - accuracy: 0.5217 - val_loss: 0.1882 - val_accuracy: 0.5577
Epoch 3/80
20/20 [==============================] - 0s 11ms/step - loss: 0.1769 - accuracy: 0.5217 - val_loss: 0.1382 - val_accuracy: 0.5577
Epoch 4/80
20/20 [==============================] - 0s 11ms/step - loss: 0.1509 - accuracy: 0.5217 - val_loss: 0.1253 - val_accuracy: 0.5577
Epoch 5/80
20/20 [==============================] - 0s 12ms/step - loss: 0.1366 - accuracy: 0.5217 - val_loss: 0.1151 - val_accuracy: 0.5577
Epoch 6/80
20/20 [==============================] - 0s 11ms/step - loss: 0.1249 - accuracy: 0.5217 - val_loss: 0.1065 - val_accuracy: 0.5577
Epoch 7/80
20/20 [==============================] - 0s 11ms/step - loss: 0.1150 - accuracy: 0.5217 - val_loss: 0.0990 - val_accuracy: 0.5577
Epoch 8/80
20

In [10]:
train_metrics = model.evaluate(
    np.array([item[0] for item in train_data]),
    np.array([item[1] for item in train_data]),
    verbose=0
)
val_metrics = model.evaluate(
    np.array([item[0] for item in val_data]),
    np.array([item[1] for item in val_data]),
    verbose=0
)
test_metrics = model.evaluate(
    np.array([item[0] for item in test_data]), 
    np.array([item[1] for item in test_data]),
    verbose=0
)

zero_return = model.predict((([0] * 13),))

for item in test_data:
    inp = np.array([item[0]])  # Reshape input data into a batch
    prediction = model.predict(inp)
    print("Input:", item[0])
    print("Expected Output:", item[1])
    print("Predicted Output:", prediction[0])


print({f"Train {metric_name}": metric_value for metric_name, metric_value in zip(model.metrics_names, train_metrics)})
print({f"Validation {metric_name}": metric_value for metric_name, metric_value in zip(model.metrics_names, val_metrics)})
print({f"Test {metric_name}": metric_value for metric_name, metric_value in zip(model.metrics_names, test_metrics)})
print("RMSE for test set:", np.sqrt(test_metrics[0]))
print(f"Zero return constants: {zero_return}")
with open(join(log_dir, "stats.md"), 'a') as f:
   f.write(f"- Zero return values: {zero_return}")

# save model as .h5, .tflite and .json
model.save(join(log_dir, "taunet.pb"))
tflite_model = tf.lite.TFLiteConverter.from_saved_model(join(log_dir, "taunet.pb")).convert()
with open(join(log_dir, "taunet.tflite"), 'wb') as f:
  f.write(tflite_model)
save_model(model, join(log_dir, "taunet.json"))

1/1 [==============================] - 0s 105ms/step
Input: [0.0, 0.0715, 0.1726, 0.588, 0.1261, 0.5281, 0.1964, 0.0314, 0.3551, 0.0, 0.5044, 0, 0.7842]
Expected Output: [0.552416, 0.4032835]
Predicted Output: [0.45583665 0.4672649 ]
1/1 [==============================] - 0s 30ms/step
Input: [0.9566, 0.1498, 0.1552, 0.8348, 0.4552, 0.482, 0.1599, 0.0237, 0.2225, 0.4043, 0.0, 0, 0.7678]
Expected Output: [0.41335340000000004, 0.11432283333333333]
Predicted Output: [0.39971474 0.42938083]
1/1 [==============================] - 0s 32ms/step
Input: [0.6843, 0.0974, 0.3054, 0.1529, 0.0348, 0.0748, 0.052, 0.0298, 0.2256, 0.0651, 0.8374, 0.6374, 0]
Expected Output: [0.18634620000000002, 0.4575912]
Predicted Output: [0.22259676 0.3098206 ]
1/1 [==============================] - 0s 28ms/step
Input: [0.8734, 0.0806, 0.0996, 0.6927, 0.4294, 0.1492, 0.1048, 0.0342, 0.1895, 0.0, 0.903, 0.46440000000000003, 0]
Expected Output: [0.5807566000000001, 0.6814665333333333]
Predicted Output: [0.5159535  0.5

INFO:tensorflow:Assets written to: logs/20240611-134441\taunet.pb\assets


INFO:tensorflow:Assets written to: logs/20240611-134441\taunet.pb\assets
